# Preprocessing 2 

The tasks completed here are as follows: 
- Load full datasets
- Remove revision tags where necessary 
- Remove specific health profiles
- Form the datasets by model size and drop rows with NaNs (more features, less observations).
- Remove rows where relevant features have value 9s (representing manually encoded missing data) 
- Save datasets

In [1]:
import sklearn 
import pandas as pd 
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 2000)

In [3]:
from pickle import dump, load

def save_pickle(path, obj): 
    with open(path, "wb") as fp:   #Pickling
        dump(obj, fp)
        
def open_pickle(path): 
    with open(path, "rb") as fp:   # Unpickling
        return load(fp)

In [4]:
# path to cleaned data 
path = "/Users/matthewkeys/Desktop/CRES/Data/SurgeryPrognostics/Cleaned/"

In [5]:
FM_knee_data = open_pickle(path+"final_knee_data.txt")
FM_hip_data = open_pickle(path+"final_hip_data.txt")
FM_groin_data = open_pickle(path+"final_groin_data.txt")

### Format Data 
In this section I create the various datasets that will be used in the below models. 
- First remove observations corresponding to revision surgeries. Prior to 2012 we have a problem.

In [6]:
# Get shapes & revision surgery counts 
for dataset in [FM_knee_data, FM_hip_data, FM_groin_data]:
    print(dataset['Revision Flag'].value_counts())
    print(dataset['Revision Flag'].isna().sum())
    print(dataset.shape)

0.0    309094
1.0     13010
Name: Revision Flag, dtype: int64
83242
(405346, 45)
0.0    284141
1.0     18281
Name: Revision Flag, dtype: int64
79654
(382076, 45)
0.0    145544
Name: Revision Flag, dtype: int64
45018
(190562, 34)


In [7]:
# Reconstruct revision surgeries for prior to 2012 (need to do...)

##################################################################


In [8]:
# Temporary solution: Impute all 1's in the revision flag for 2010 and 2011
FM_knee_data['Revision Flag'] = FM_knee_data.apply(
    lambda row: 0 if ((np.isnan(row['Revision Flag'])) & ((row['Year'] == 2010) | (row['Year'] == 2011))) 
    else row['Revision Flag'], axis=1)

FM_hip_data['Revision Flag'] = FM_hip_data.apply(
    lambda row: 0 if ((np.isnan(row['Revision Flag'])) & ((row['Year'] == 2010) | (row['Year'] == 2011))) 
    else row['Revision Flag'], axis=1)

FM_groin_data['Revision Flag'] = FM_groin_data.apply(
    lambda row: 0 if ((np.isnan(row['Revision Flag'])) & ((row['Year'] == 2010) | (row['Year'] == 2011))) 
    else row['Revision Flag'], axis=1)

In [9]:
# Remove revision surgeries from datasets
FM_knee_data = FM_knee_data.loc[FM_knee_data['Revision Flag'] == 0, :]
FM_hip_data = FM_hip_data.loc[FM_hip_data['Revision Flag'] == 0, :]
FM_groin_data = FM_groin_data.loc[FM_groin_data['Revision Flag'] == 0, :]

In [10]:
# Remove health profiles 11111 and 33333 (cannot deteroriate or improve)
FM_knee_data = FM_knee_data.loc[(FM_knee_data['Pre-Op Q EQ5D Index Profile']
                                != 11111) & (FM_knee_data['Pre-Op Q EQ5D Index Profile'] 
                                != 33333)]
FM_hip_data = FM_hip_data.loc[(FM_hip_data['Pre-Op Q EQ5D Index Profile']
                                != 11111) & (FM_hip_data['Pre-Op Q EQ5D Index Profile'] 
                                != 33333)]
FM_groin_data = FM_groin_data.loc[(FM_groin_data['Pre-Op Q EQ5D Index Profile']
                                != 11111) & (FM_groin_data['Pre-Op Q EQ5D Index Profile'] 
                                != 33333)]

###### Reduced Model Data
Limit features (pre-op characteristics) to age, gender, baseline score and symptom duration. 

In [11]:
# Reduced Model (RM) knee data 
RM_knee_data = FM_knee_data[['Age Band', 'Gender', 'Pre-Op Q EQ5D Index',
                            'Pre-Op Q Symptom Period', 'Post-Op Q EQ5D Index']]

In [12]:
# Reduced Model (RM) hip data 
RM_hip_data = FM_hip_data[['Age Band', 'Gender', 'Pre-Op Q EQ5D Index',
                            'Pre-Op Q Symptom Period', 'Post-Op Q EQ5D Index']]

In [13]:
# Reduced Model (RM) groin data 
RM_groin_data = FM_groin_data[['Age Band', 'Gender', 'Pre-Op Q EQ5D Index',
                            'Pre-Op Q Symptom Period', 'Post-Op Q EQ5D Index']]

In [14]:
# Get NA counts. Why so many missing values? 
for dataset in [RM_knee_data, RM_hip_data, RM_groin_data]:
    print(dataset.shape)
    print(np.count_nonzero(dataset.isnull().values))
    print(dataset.isna().sum())

(391023, 5)
165847
Age Band                        0
Gender                          0
Pre-Op Q EQ5D Index         22676
Pre-Op Q Symptom Period    122406
Post-Op Q EQ5D Index        20765
dtype: int64
(362715, 5)
153933
Age Band                        0
Gender                          0
Pre-Op Q EQ5D Index         20224
Pre-Op Q Symptom Period    115237
Post-Op Q EQ5D Index        18472
dtype: int64
(140680, 5)
61582
Age Band                       0
Gender                         0
Pre-Op Q EQ5D Index         5069
Pre-Op Q Symptom Period    49728
Post-Op Q EQ5D Index        6785
dtype: int64


In [15]:
# Remove NAs    
RM_knee_data = RM_knee_data.dropna()
RM_hip_data = RM_hip_data.dropna() 
RM_groin_data = RM_groin_data.dropna() 

In [16]:
for dataset in [RM_knee_data, RM_hip_data, RM_groin_data]:
    print(dataset.shape)

(242485, 5)
(224228, 5)
(83607, 5)


###### Full Model Data (Original) 
This limits the features to all pre-operative charateristics described in the dataset, as in the original paper. These vary between each dataset. Basically we just remove the comorbidities.

In [17]:
FMO_knee_data = FM_knee_data.drop(['Post-Op Q EQ5D Index Profile', 'Revision Flag',
                                   'Arthritis', 'Cancer', 'Circulation', 'Depression',
                                   'Diabetes', 'Heart Disease', 'High Bp', 'Kidney Disease',
                                   'Liver Disease', 'Lung Disease', 'Nervous System', 'Stroke']
                                  , axis=1)

In [18]:
FMO_hip_data = FM_hip_data.drop(['Post-Op Q EQ5D Index Profile', 'Revision Flag',
                                   'Arthritis', 'Cancer', 'Circulation', 'Depression',
                                   'Diabetes', 'Heart Disease', 'High Bp', 'Kidney Disease',
                                   'Liver Disease', 'Lung Disease', 'Nervous System', 'Stroke']
                                  , axis=1)

In [19]:
FMO_groin_data = FM_groin_data.drop(['Post-Op Q EQ5D Index Profile', 'Revision Flag',
                                   'Arthritis', 'Cancer', 'Circulation', 'Depression',
                                   'Diabetes', 'Heart Disease', 'High Bp', 'Kidney Disease',
                                   'Liver Disease', 'Lung Disease', 'Nervous System', 'Stroke']
                                  , axis=1)

In [20]:
for dataset in [FMO_knee_data, FMO_hip_data, FMO_groin_data]:
    print(dataset.shape)
    print(np.count_nonzero(dataset.isnull().values))
    print(dataset.isna().sum())

(391023, 31)
555748
Age Band                                      0
Gender                                        0
Pre-Op Q Symptom Period                  122406
Knee Replacement Post-Op Q Score           7689
Knee Replacement Pre-Op Q Score            4259
Post-Op Q EQ5D Index                      20765
Pre-Op Q EQ5D Index                       22676
Pre-Op Q EQ5D Index Profile                 786
Knee Replacement Pre-Op Q Confidence          0
Knee Replacement Pre-Op Q Kneeling            0
Knee Replacement Pre-Op Q Limping             0
Knee Replacement Pre-Op Q Night Pain          0
Knee Replacement Pre-Op Q Pain                0
Knee Replacement Pre-Op Q Shopping            0
Knee Replacement Pre-Op Q Stairs              0
Knee Replacement Pre-Op Q Standing            0
Knee Replacement Pre-Op Q Transport           0
Knee Replacement Pre-Op Q Walking             0
Knee Replacement Pre-Op Q Washing             0
Knee Replacement Pre-Op Q Work                0
Pre-Op Q Activity   

In [21]:
FMO_knee_data = FMO_knee_data.dropna()
FMO_hip_data = FMO_hip_data.dropna()
FMO_groin_data = FMO_groin_data.dropna()

In [22]:
for dataset in [FMO_knee_data, FMO_hip_data, FMO_groin_data]:
    print(dataset.shape)

(158831, 31)
(148142, 31)
(57083, 20)


###### Full Model Data (Adapted)
These datasets use more features at the cost of less observations. In particular, over the original feature set, we also include the various comorbidities. These include Arthritis, Cancer, Circulation, Depression, Diabetes, Heart Disease, High Bp, Kidney Disease,	Liver Disease, Lung Disease, Nervous System, and Stroke. 

In [23]:
FMA_knee_data = FM_knee_data.drop(['Post-Op Q EQ5D Index Profile', 'Revision Flag']
                                  , axis=1)

In [24]:
FMA_hip_data = FM_hip_data.drop(['Post-Op Q EQ5D Index Profile', 'Revision Flag']
                                , axis=1)

In [25]:
FMA_groin_data = FM_groin_data.drop(['Post-Op Q EQ5D Index Profile', 'Revision Flag']
                                    , axis=1)

In [26]:
for dataset in [FMA_knee_data, FMA_hip_data, FMA_groin_data]:
    print(dataset.shape)
    print(np.count_nonzero(dataset.isnull().values))
    print(dataset.isna().sum())

(391023, 43)
2024620
Age Band                                      0
Gender                                        0
Arthritis                                122406
Cancer                                   122406
Circulation                              122406
Depression                               122406
Diabetes                                 122406
Heart Disease                            122406
High Bp                                  122406
Kidney Disease                           122406
Liver Disease                            122406
Lung Disease                             122406
Nervous System                           122406
Stroke                                   122406
Pre-Op Q Symptom Period                  122406
Knee Replacement Post-Op Q Score           7689
Knee Replacement Pre-Op Q Score            4259
Post-Op Q EQ5D Index                      20765
Pre-Op Q EQ5D Index                       22676
Pre-Op Q EQ5D Index Profile                 786
Knee Replacement Pr

In [27]:
# Drop NAs 
FMA_knee_data = FMA_knee_data.dropna()
FMA_hip_data = FMA_hip_data.dropna()
FMA_groin_data = FMA_groin_data.dropna()

In [28]:
for dataset in [FMA_knee_data, FMA_hip_data, FMA_groin_data]:
    print(dataset.shape)

(158831, 43)
(148142, 43)
(57083, 32)


In [29]:
# Compare lengths of all datasets 
for dataset in [RM_knee_data, RM_hip_data, RM_groin_data]:
    print(dataset.shape)
    
for dataset in [FMO_knee_data, FMO_hip_data, FMO_groin_data]:
    print(dataset.shape)
    
for dataset in [FMA_knee_data, FMA_hip_data, FMA_groin_data]:
    print(dataset.shape)

(242485, 5)
(224228, 5)
(83607, 5)
(158831, 31)
(148142, 31)
(57083, 20)
(158831, 43)
(148142, 43)
(57083, 32)


###### Correct Encoding
Utilising the PROMs data dictionary, this section focuses on correcting the encoding of the categorical variables. Despite their numerical format, several of the variables are indeed categorical. I.e. a 9 does not represent a more intense disease state than a 0 or a 1, but rather incomplete data for example. Problems: 
- All of the diagnosed comorbidities are encoded (1,9), where 9 represents missing data and 1 represents a yes. Why can we not distinguish between not having cancer and missing info? Can we possibly turn this into a classification problem? Generally may not be a problem.
- For all the remaining listed variables, 9 indicates only missing data, and so these observations can be removed.

In [30]:
# Example:
print(FMA_knee_data["Pre-Op Q Symptom Period"].unique())
print(FMA_hip_data["Pre-Op Q Symptom Period"].unique())
print(FMA_groin_data["Pre-Op Q Symptom Period"].unique())
# 9 appears in all the data despite not being in the data dictionary

[2. 3. 4. 1. 9.]
[4. 2. 1. 3. 9.]
[1. 2. 9.]


In [31]:
# None of the EQ5D dimensions have missing values, and we want to keep 9s in the
# comorbidities, as it does not represent systematic measurement error and may still 
# be useful features. 
for dataset in [FMA_knee_data, FMA_hip_data, FMA_groin_data]: 
    for column in ["Pre-Op Q Activity", "Pre-Op Q Anxiety", "Pre-Op Q Discomfort",
                   "Pre-Op Q Mobility", "Pre-Op Q Self-Care", "Pre-Op Q EQ5D Index Profile",
                   "Post-Op Q EQ5D Index"]:
        print(dataset[column].unique())

[2 3 1]
[2 3 1]
[3 2 1]
[2 1 3]
[2 1 3]
[22232. 22333. 21231. 21331. 21232. 22221. 21221. 21222. 22331. 21131.
 22222. 21121. 11221. 11121. 22233. 22231. 22332. 21333. 21332. 21321.
 21322. 21132. 21122. 21323. 22223. 22322. 11211. 21223. 11233. 22131.
 12231. 11222. 11231. 21112. 11212. 12221. 32333. 21233. 21211. 22321.
 11131. 11321. 21123. 11122. 23332. 21111. 12322. 11132. 11232. 22211.
 23221. 11112. 33332. 23331. 32231. 11322. 23231. 23333. 22323. 22121.
 31331. 23222. 23233. 32331. 32322. 12232. 22122. 23322. 22212. 32332.
 12211. 21212. 23321. 31233. 21213. 23122. 23232. 13221. 12222. 11223.
 11113. 31221. 32232. 32233. 12331. 22311. 33331. 21311. 11133. 12321.
 11123. 31121. 11331. 11311. 11323. 12121. 22132. 32323. 11332. 22312.
 21133. 22123. 13211. 12122. 12332. 12233. 22133. 21312. 32321. 21313.
 31321. 11333. 33322. 31222. 31131. 12132. 33231. 12112. 22111. 31231.
 12333. 32222. 22213. 32223. 23311. 23121. 12212. 12111. 23323. 31333.
 33232. 31323. 23131. 23123. 13232. 2

In [32]:
# Columns to check for the presence of 9s (NAs) and consequently remove observations
KR9C = ['Pre-Op Q Symptom Period', 'Knee Replacement Pre-Op Q Confidence', 
       'Knee Replacement Pre-Op Q Kneeling', 'Knee Replacement Pre-Op Q Limping',
       'Knee Replacement Pre-Op Q Night Pain', 'Knee Replacement Pre-Op Q Pain', 
       'Knee Replacement Pre-Op Q Shopping', 'Knee Replacement Pre-Op Q Stairs', 
       'Knee Replacement Pre-Op Q Standing', 'Knee Replacement Pre-Op Q Transport', 
       'Knee Replacement Pre-Op Q Walking', 'Knee Replacement Pre-Op Q Washing',
       'Knee Replacement Pre-Op Q Work']

HR9C = ['Hip Replacement Pre-Op Q Dressing', 'Hip Replacement Pre-Op Q Limping', 
        'Hip Replacement Pre-Op Q Night Pain', 'Hip Replacement Pre-Op Q Pain', 
        'Hip Replacement Pre-Op Q Shopping', 'Hip Replacement Pre-Op Q Stairs', 
        'Hip Replacement Pre-Op Q Standing', 'Hip Replacement Pre-Op Q Sudden Pain', 
        'Hip Replacement Pre-Op Q Transport', 'Hip Replacement Pre-Op Q Walking', 
        'Hip Replacement Pre-Op Q Washing', 'Hip Replacement Pre-Op Q Work',
        'Pre-Op Q Symptom Period']

GH9C = ['Pre-Op Q Symptom Period', 'Pre-Op Q Disability', 
        'Pre-Op Q Assisted', 'Pre-Op Q Assisted By'] 

# Possible interaction between Assisted and Assisted by (multicollinearity). 

In [33]:
# Remove observations with 9s in the appropriate columns
FMO_knee_data = FMO_knee_data[(FMO_knee_data[KR9C] != 9).all(axis=1)]
FMO_hip_data = FMO_hip_data[(FMO_hip_data[HR9C] != 9).all(axis=1)]
FMO_groin_data = FMO_groin_data[(FMO_groin_data[GH9C] != 9).all(axis=1)]

In [38]:
FMA_knee_data = FMA_knee_data[(FMA_knee_data[KR9C] != 9).all(axis=1)]
FMA_hip_data = FMA_hip_data[(FMA_hip_data[HR9C] != 9).all(axis=1)]
FMA_groin_data = FMA_groin_data[(FMA_groin_data[GH9C] != 9).all(axis=1)]

In [49]:
# Adapt the columns to check 
KR9CC = KR9C[0:1]
HR9CC = HR9C[12:13]
GH9CC = GH9C[0:1]

In [50]:
RM_knee_data = RM_knee_data[(RM_knee_data[KR9CC] != 9).all(axis=1)]
RM_hip_data = RM_hip_data[(RM_hip_data[HR9CC] != 9).all(axis=1)]
RM_groin_data = RM_groin_data[(RM_groin_data[GH9CC] != 9).all(axis=1)]

In [53]:
# Convert '*'s in otherwise numerical columns to numeric 
# FMA
FMA_knee_data['Gender'] = FMA_knee_data['Gender'].convert_objects(convert_numeric=True)
FMA_knee_data = FMA_knee_data.dropna()
print(FMA_knee_data.shape)

/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


(147031, 43)


In [54]:
FMA_hip_data['Gender'] = FMA_hip_data['Gender'].convert_objects(convert_numeric=True)
FMA_hip_data = FMA_hip_data.dropna()
print(FMA_hip_data.shape)

/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


(135719, 43)


In [55]:
FMA_groin_data['Gender'] = FMA_groin_data['Gender'].convert_objects(convert_numeric=True)
FMA_groin_data = FMA_groin_data.dropna()
print(FMA_groin_data.shape)

(46023, 32)


/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [56]:
# FMO
FMO_knee_data['Gender'] = FMO_knee_data['Gender'].convert_objects(convert_numeric=True)
FMO_knee_data = FMO_knee_data.dropna()
print(FMO_knee_data.shape)

/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


(147031, 31)


In [57]:
FMO_hip_data['Gender'] = FMO_hip_data['Gender'].convert_objects(convert_numeric=True)
FMO_hip_data = FMO_hip_data.dropna()
print(FMO_hip_data.shape)

(135719, 31)


/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [58]:
FMO_groin_data['Gender'] = FMO_groin_data['Gender'].convert_objects(convert_numeric=True)
FMO_groin_data = FMO_groin_data.dropna()
print(FMO_groin_data.shape)

(46023, 20)


/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [59]:
# RM 
RM_knee_data['Gender'] = RM_knee_data['Gender'].convert_objects(convert_numeric=True)
RM_knee_data = RM_knee_data.dropna()
print(RM_knee_data.shape)

(226105, 5)


/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [60]:
RM_hip_data['Gender'] = RM_hip_data['Gender'].convert_objects(convert_numeric=True)
RM_hip_data = RM_hip_data.dropna()
print(RM_hip_data.shape)

(207191, 5)


/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [61]:
RM_groin_data['Gender'] = RM_groin_data['Gender'].convert_objects(convert_numeric=True)
RM_groin_data = RM_groin_data.dropna()
print(RM_groin_data.shape)

(70219, 5)


/Users/matthewkeys/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


### Save Datasets
- Save datasets for transfer to R for modelling. Use feather package.

In [62]:
import feather

In [63]:
path = "/Users/matthewkeys/Desktop/CRES/Data/SurgeryPrognostics/Cleaned/"

In [64]:
feather.write_dataframe(RM_knee_data, path+"reduced_knee_data.feather")
feather.write_dataframe(RM_hip_data, path+"reduced_hip_data.feather")
feather.write_dataframe(RM_groin_data, path+"reduced_groin_data.feather")

In [65]:
feather.write_dataframe(FMO_knee_data, path+"large_knee_data.feather")
feather.write_dataframe(FMO_hip_data, path+"large_hip_data.feather")
feather.write_dataframe(FMO_groin_data, path+"large_groin_data.feather")

In [66]:
feather.write_dataframe(FMA_knee_data, path+"larger_knee_data.feather")
feather.write_dataframe(FMA_hip_data, path+"larger_hip_data.feather")
feather.write_dataframe(FMA_groin_data, path+"larger_groin_data.feather")